In [1]:
import pyranges as pr
import pandas as pd

In [2]:
%time
gtf = pr.read_gtf("../data/reference_filtered.gtf")

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 5.01 µs


In [3]:
le_gtf = pr.read_gtf("../processed/2023-06-20_cryptic_last_exons.gtf")

In [4]:
# minimal info required from GTF - gene_id (representing REF gene_id) & transcript id
# only need to add novel last exons
le_gtf_min = (le_gtf.subset(lambda df: df.annot_status == "novel")
 [["Feature", "Source", "Score", "ref_gene_id", "ref_gene_name", "transcript_id", "annot_status"]]
 .apply(lambda df: df.rename(columns={"ref_gene_id": "gene_id", "ref_gene_name": "gene_name"}))
 .drop("annot_status"))

le_gtf_min


,Chromosome,Feature,Source,Start,End,Score,Strand,gene_id,gene_name,transcript_id
0,chr1,exon,.,15939295,15940477,.,+,ENSG00000065526.12,SPEN,PAPA.chx_tdp_DOX_ctrl_2.224.5
1,chr1,exon,.,40421975,40423342,.,+,ENSG00000084070.12,SMAP2,PAPA.TDP-6.631.2
2,chr1,exon,.,147622270,147623356,.,+,ENSG00000116128.12,BCL9,PAPA.doxconc_DOX_0075_2.1123.3
3,chr1,exon,.,147622270,147622693,.,+,ENSG00000116128.12,BCL9,PAPA.chx_tdp_DOX_ctrl_2.1186.4
4,chr1,exon,.,147622270,147623701,.,+,ENSG00000116128.12,BCL9,PAPA.doxconc_DOX_0075_1.1139.1
...,...,...,...,...,...,...,...,...,...,...
865,chrX,exon,.,17835910,17837395,.,-,ENSG00000131831.18,"RAI2,RAI2",PAPA.TDP43-G_S7.20262.6
866,chrX,exon,.,40653641,40654556,.,-,ENSG00000180182.11,MED14,PAPA.TDP43-G_S7.20350.2
867,chrX,exon,.,40653676,40654556,.,-,ENSG00000180182.11,MED14,PAPA.TDP43_ctrl_3.26410.3
868,chrX,exon,.,131823775,131825365,.,-,ENSG00000213468.7,FIRRE,PAPA.TDP43-F_S6.20505.8


In [5]:
# sometimes I have gene_names duplicated - remove duplicates, returning to comma-separatd string in same insertion order as before in case still unqiue values
le_gtf_min.gene_name = le_gtf_min.gene_name.apply(lambda x:  ",".join(list(dict.fromkeys(x.split(",")))))
le_gtf_min

,Chromosome,Feature,Source,Start,End,Score,Strand,gene_id,gene_name,transcript_id
0,chr1,exon,.,15939295,15940477,.,+,ENSG00000065526.12,SPEN,PAPA.chx_tdp_DOX_ctrl_2.224.5
1,chr1,exon,.,40421975,40423342,.,+,ENSG00000084070.12,SMAP2,PAPA.TDP-6.631.2
2,chr1,exon,.,147622270,147623356,.,+,ENSG00000116128.12,BCL9,PAPA.doxconc_DOX_0075_2.1123.3
3,chr1,exon,.,147622270,147622693,.,+,ENSG00000116128.12,BCL9,PAPA.chx_tdp_DOX_ctrl_2.1186.4
4,chr1,exon,.,147622270,147623701,.,+,ENSG00000116128.12,BCL9,PAPA.doxconc_DOX_0075_1.1139.1
...,...,...,...,...,...,...,...,...,...,...
865,chrX,exon,.,17835910,17837395,.,-,ENSG00000131831.18,RAI2,PAPA.TDP43-G_S7.20262.6
866,chrX,exon,.,40653641,40654556,.,-,ENSG00000180182.11,MED14,PAPA.TDP43-G_S7.20350.2
867,chrX,exon,.,40653676,40654556,.,-,ENSG00000180182.11,MED14,PAPA.TDP43_ctrl_3.26410.3
868,chrX,exon,.,131823775,131825365,.,-,ENSG00000213468.7,FIRRE,PAPA.TDP43-F_S6.20505.8


In [7]:
# combine the two gtfs
comb_gtf = pr.concat([gtf, le_gtf_min])
comb_gtf

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,hgnc_id,protein_id,ccdsid
0,chr1,HAVANA,transcript,358871,365510,.,+,.,ENSG00000236601.2,lncRNA,...,lncRNA,ENST00000412666,1.0,not_best_in_genome_evidence,OTTHUMT00000007987.2,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,exon,358871,358957,.,+,.,ENSG00000236601.2,lncRNA,...,lncRNA,ENST00000412666,1.0,not_best_in_genome_evidence,OTTHUMT00000007987.2,1,ENSE00001700697.1,NaN,NaN,NaN
2,chr1,HAVANA,exon,365170,365510,.,+,.,ENSG00000236601.2,lncRNA,...,lncRNA,ENST00000412666,1.0,not_best_in_genome_evidence,OTTHUMT00000007987.2,2,ENSE00001745103.1,NaN,NaN,NaN
3,chr1,HAVANA,transcript,360056,366052,.,+,.,ENSG00000236601.2,lncRNA,...,lncRNA,ENST00000635159,1.0,"non_submitted_evidence,basic",OTTHUMT00000488661.1,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,exon,360056,360168,.,+,.,ENSG00000236601.2,lncRNA,...,lncRNA,ENST00000635159,1.0,"non_submitted_evidence,basic",OTTHUMT00000488661.1,1,ENSE00003784495.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556647,chrY,HAVANA,exon,25386916,25387085,.,-,.,ENSG00000228786.5,lncRNA,...,lncRNA,LINC00266-4P-201,3.0,"basic,Ensembl_canonical",OTTHUMT00000104946.1,2,ENSE00001653980.1,HGNC:38800,NaN,NaN
1556648,chrY,HAVANA,exon,25378299,25378878,.,-,.,ENSG00000228786.5,lncRNA,...,lncRNA,LINC00266-4P-201,3.0,"basic,Ensembl_canonical",OTTHUMT00000104946.1,3,ENSE00001596444.1,HGNC:38800,NaN,NaN
1556649,chrY,HAVANA,transcript,57201142,57203357,.,-,.,ENSG00000185203.12_PAR_Y,lncRNA,...,lncRNA,WASIR1-201,1.0,"not_best_in_genome_evidence,basic,Ensembl_cano...",OTTHUMT00000058824.3,NaN,NaN,HGNC:38513,NaN,NaN
1556650,chrY,HAVANA,exon,57203181,57203357,.,-,.,ENSG00000185203.12_PAR_Y,lncRNA,...,lncRNA,WASIR1-201,1.0,"not_best_in_genome_evidence,basic,Ensembl_cano...",OTTHUMT00000058824.3,1,ENSE00001772131.3,HGNC:38513,NaN,NaN


In [8]:
# generate tx2gene for subsequent merging
tx2gene2name = comb_gtf.as_df()[["transcript_id", "gene_id", "gene_name"]].drop_duplicates()
tx2gene2name

,transcript_id,gene_id,gene_name
0,ENST00000412666.1,ENSG00000236601.2,ENSG00000236601
3,ENST00000635159.1,ENSG00000236601.2,ENSG00000236601
6,ENST00000450696.1,ENSG00000235146.2,ENSG00000235146
9,ENST00000412115.2,ENSG00000237491.10,LINC01409
15,ENST00000429505.6,ENSG00000237491.10,LINC01409
...,...,...,...
1556622,ENST00000618574.1,ENSG00000185894.8,BPY2C
1556634,ENST00000456123.1,ENSG00000228296.1,TTTY4C
1556639,ENST00000421387.2,ENSG00000223641.2,TTTY17C
1556645,ENST00000427373.5,ENSG00000228786.5,LINC00266-4P


In [ ]:
comb_gtf.to_gtf("../processed/2023-08-23_reference_filtered_cryptic_le_combined.gtf")
tx2gene2name.to_csv("../processed/2023-08-23_reference_filtered_cryptic_le_combined.tx2gene2name.tsv", sep="\t", header=True, index=False)